### 2.1 Importing Libraries and Reading Files 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm_notebook as tqdm
import re
from string import punctuation
from prettytable import PrettyTable
import math
pd.set_option("display.max_columns",None)

data=pd.read_csv('train.csv',low_memory=False)
np.random.seed(43)

In [2]:
def preprocess():
    
    '''Preprocess and clean some of the data, assigning  values to missing values based on the feature and updating Team Abbrevations to remain consistent 
       Returns the list of columns with empty features'''
    
    orient=data.groupby('Position',as_index=False)['Orientation'].mean()
    Dir=data.groupby('Position',as_index=False)['Dir'].mean()
    OrientDict={}
    DirDict={}
    for i,j in orient.iterrows():
        OrientDict[j['Position']]=j['Orientation']
        DirDict[j['Position']]=j['Orientation']

    data.Dir=data.Dir.fillna(data.Position.map(DirDict))
    data.Orientation=data.Orientation.fillna(data.Position.map(OrientDict))

    tempsperweek=data.groupby(['Season','Week'],as_index=False)['Temperature'].mean()
    temp2017={}
    temp2018={}
    temp2019={}
    for i,j in tempsperweek.iterrows():
        if j['Season']==2017:
            temp2017[j['Week']]=j['Temperature']
        if j['Season']==2018:
            temp2018[j['Week']]=j['Temperature']
        if j['Season']==2019:
            temp2019[j['Week']]=j['Temperature']

    data.loc[(data.Season==2017) & (data.Temperature.isna()),'Temperature']=data.Temperature.fillna(data.Week.map(temp2017))
    data.loc[(data.Season==2018) & (data.Temperature.isna()),'Temperature']=data.Temperature.fillna(data.Week.map(temp2018))
    data.loc[(data.Season==2019) & (data.Temperature.isna()),'Temperature']=data.Temperature.fillna(data.Week.map(temp2019))

    data.loc[data.FieldPosition.isna(),'FieldPosition']='UNKNOWN'
    data.loc[data.OffenseFormation.isna(),'OffenseFormation']=data.OffenseFormation.value_counts().idxmax()

    data.loc[data.DefendersInTheBox.isna(),'DefendersInTheBox']=data.DefendersInTheBox.median()
    data.loc[data.Humidity.isna(),'Humidity']=data.Humidity.median()


    #Code to clean StadiumType 
    #code inspired from https://www.kaggle.com/sanshengshi/lightgbm-clean-stadiumtype
    def StadiumType(txt):
        txt=str(txt)
        txt=txt.lower()
        txt=txt.strip()
        if 'indoor' in txt or 'closed' in txt:
            return 0
        else:
            return 1   #outdoor or open or unspecified is being treated an as open field 
    data["StadiumType"]=data["StadiumType"].apply(StadiumType)

    def Gameweather(txt):
        txt=str(txt)
        txt=txt.lower()
        txt=txt.strip()
        if 'clear' in txt or 'sun' in txt or 'controlled' in txt or 'indoor' in txt:
            return 0
        if 'rain' in txt:
            return 1
        if 'cloud' in txt or 'overcast' in txt:
            return 0.5 
        if 'snow' in txt or 'overcast' in txt:
            return -0.5
        return 0                                   # Values given to differentiate between clear and rainy 
    data["GameWeather"]=data["GameWeather"].apply(Gameweather)

    def Windspeed(txt):
        if pd.isna(txt):
            return 7.0                   # Median Value   
        if '-' in txt:
            a,b=txt.split('-')
            return (float(a)+float(b))/2
        elif txt.isalnum():
            if re.match('(\d+)',str(txt)):
                return float(re.match('(\d+)',str(txt))[0])
            else:
                return 7.0
        else:
            return 0
    data["WindSpeed"]=data["WindSpeed"].astype(str)
    data["WindSpeed"]=data["WindSpeed"].apply(Windspeed)


    # code based from https://www.kaggle.com/bgmello/neural-networks-feature-engineering-for-the-win
    def WindDirection(txt):

        #Cleaning the values
        if pd.isna(txt):
            return -1
        txt = txt.lower()
        txt = ''.join([c for c in txt if c not in punctuation])
        txt = txt.replace('from', '')
        txt = txt.replace(' ', '')
        txt = txt.replace('north', 'n')
        txt = txt.replace('south', 's')
        txt = txt.replace('west', 'w')
        txt = txt.replace('east', 'e')

        #assigning the values

        deg=360
        if txt=='n':
            return 0
        if txt=='nne' or txt=='nen':
            return 1/16*deg
        if txt=='ne':
            return 2/16*deg
        if txt=='ene' or txt=='nee':
            return 3/16*deg
        if txt=='e':
            return 4/16*deg
        if txt=='ese' or txt=='see':
            return 5/16*deg
        if txt=='se':
            return 6/16*deg
        if txt=='ses' or txt=='sse':
            return 7/16*deg
        if txt=='s':
            return 8/16*deg
        if txt=='ssw' or txt=='sws':
            return 9/16*deg
        if txt=='sw':
            return 10/16*deg
        if txt=='sww' or txt=='wsw':
            return 11/16*deg
        if txt=='w':
            return 12/16*deg
        if txt=='wnw' or txt=='nww':
            return 13/16*deg
        if txt=='nw':
            return 14/16*deg
        if txt=='nwn' or txt=='nnw':
            return 15/16*deg
        return -1
                                          # Values given to differentiate between clear and rainy 
    data["WindDirection"]=data["WindDirection"].apply(WindDirection)
    
    TeamMap={'ARI':'ARZ','BAL':'BLT','CLE':'CLV','HOU':'HST'}

    for k,v in TeamMap.items():
        data.loc[data['VisitorTeamAbbr']==k,'VisitorTeamAbbr']=v
        data.loc[data['HomeTeamAbbr']==k,'HomeTeamAbbr']=v
    data['PlayId']=data['PlayId'].astype(str)
    data['PlayerHeight']=data['PlayerHeight'].astype(str)
    data['PlayerHeight']=data['PlayerHeight'].str.split('-').apply(lambda x: int(x[0])*0.3048+ int(x[1])*0.0254)
    data['PlayerWeight']=data['PlayerWeight']*0.453592
    
    #first 3 Downs have similar distribution
    data['Down']=data.Down.apply(lambda x: 1 if x<3 else 0)
    
    return data.columns[data.isna().any()==True].values

In [3]:
%%time
preprocess()

Wall time: 6.65 s


array([], dtype=object)

In [4]:
data

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,DisplayName,JerseyNumber,Season,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,PlayerHeight,PlayerWeight,PlayerBirthDate,PlayerCollegeName,Position,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection
0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,496723,Eric Berry,29,2017,35,1,14:14:00,NE,0,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,1.8288,96.161504,12/29/1988,Tennessee,SS,NE,KC,1,Gillette Stadium,"Foxborough, MA",1,Field Turf,0.0,63.0,77.0,8.0,225.0
1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,2495116,Allen Bailey,97,2017,35,1,14:14:00,NE,0,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,1.9050,130.634496,03/25/1989,Miami,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",1,Field Turf,0.0,63.0,77.0,8.0,225.0
2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,2495493,Justin Houston,50,2017,35,1,14:14:00,NE,0,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,1.9050,122.469840,01/21/1989,Georgia,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",1,Field Turf,0.0,63.0,77.0,8.0,225.0
3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,2506353,Derrick Johnson,56,2017,35,1,14:14:00,NE,0,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,1.9050,111.130040,11/22/1982,Texas,ILB,NE,KC,1,Gillette Stadium,"Foxborough, MA",1,Field Turf,0.0,63.0,77.0,8.0,225.0
4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,2530794,Ron Parker,38,2017,35,1,14:14:00,NE,0,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,1.8288,93.439952,08/17/1987,Newberry,FS,NE,KC,1,Gillette Stadium,"Foxborough, MA",1,Field Turf,0.0,63.0,77.0,8.0,225.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682149,2019112500,20191125003789,home,69.58,31.33,4.19,1.33,0.43,92.32,20.94,2556593,Cory Littleton,58,2019,38,4,04:11:00,BLT,1,9,BLT,6,45,2562407,PISTOL,"1 RB, 1 TE, 3 WR",6.0,"3 DL, 3 LB, 5 DB",left,2019-11-26T03:59:06.000Z,2019-11-26T03:59:05.000Z,4,1.9050,103.418976,11/18/1993,Washington,ILB,LA,BLT,12,Los Angeles Memorial Coliseum,"Los Angeles, CA",1,Grass,0.0,62.0,64.0,5.0,247.5
682150,2019112500,20191125003789,home,72.16,25.09,1.01,1.40,0.08,65.83,76.48,2558199,Tanzel Smart,92,2019,38,4,04:11:00,BLT,1,9,BLT,6,45,2562407,PISTOL,"1 RB, 1 TE, 3 WR",6.0,"3 DL, 3 LB, 5 DB",left,2019-11-26T03:59:06.000Z,2019-11-26T03:59:05.000Z,4,1.8542,133.809640,11/06/1994,Tulane,DT,LA,BLT,12,Los Angeles Memorial Coliseum,"Los Angeles, CA",1,Grass,0.0,62.0,64.0,5.0,247.5
682151,2019112500,20191125003789,home,72.23,31.95,1.75,0.68,0.17,93.99,90.26,2558830,Samson Ebukam,50,2019,38,4,04:11:00,BLT,1,9,BLT,6,45,2562407,PISTOL,"1 RB, 1 TE, 3 WR",6.0,"3 DL, 3 LB, 5 DB",left,2019-11-26T03:59:06.000Z,2019-11-26T03:59:05.000Z,4,1.9050,111.130040,05/09/1995,Eastern Washington,OLB,LA,BLT,12,Los Angeles Memorial Coliseum,"Los Angeles, CA",1,Grass,0.0,62.0,64.0,5.0,247.5
682152,2019112500,20191125003789,home,72.08,26.74,0.70,1.59,0.07,61.47,48.34,2562391,Greg Gaines,91,2019,38,4,04:11:00,BLT,1,9,BLT,6,45,2562407,PISTOL,"1 RB, 1 TE, 3 WR",6.0,"3 DL, 3 LB, 5 DB",left,2019-11-26T03:59:06.000Z,2019-11-26T03:59:05.000Z,4,1.8542,141.520704,05/06/1996,Washington,DT,LA,BLT,12,Los

In [5]:
def standard_play():
    '''Function to standardise directions and other related variables'''
    
    data['Left']=data.PlayDirection=='left'
    
    data['X_std']=data.X
    data.loc[data.Left,'X_std']=120-data.loc[data.Left,'X']
    data['Y_std']=data.Y
    data.loc[data.Left,'Y_std']=160/3-data.loc[data.Left,'Y']
    
    #Rusher varibale to indicate the rushing player
    data['Rusher']=data.NflId==data.NflIdRusher
    
    data['Offense'] = "home"
    # If attacking team is not home team, it is treated as away team
    data.loc[data.PossessionTeam != data.HomeTeamAbbr, 'Offense'] = "away"
    # If Field position and possession team are same then the Yardline values stay same. Otherwise it is in the opposite direction
    data['YardLine_std'] = 100 - data.YardLine
    data.loc[data.FieldPosition == data.PossessionTeam,'YardLine_std'] = data.loc[data.FieldPosition == data.PossessionTeam,'YardLine']
    
    # Degrees to Radians subtracting by 90 since 0 degrees is upwards in raw data and it doesn't make intuitve sense, 
    # towards right is gven as 0 and towards left as 180 degrees or pi radians
    data['Dir_std']=np.mod(90-data.Dir,360)*np.pi/180
    #Flipping direction of all Left direction by pi radians (180 degrees)
    data.loc[data['Left'],'Dir_std']=np.mod(np.pi-data.loc[data['Left'],'Dir_std'],2*np.pi)
    
    data.drop(['X','Y','Dir','YardLine'],axis=1,inplace=True)
    return

standard_play()

## 4. Feature Engineering

In [6]:
#Values taken just from looking at the distrubition of data after comparing 2017 with other Seasons(2018/19)
#Number of samples also taken after looking at the corresponding values in 2018/2019 data respectively 
index=data.loc[(data.Season==2017) & (data.Dis==0.0)].sample(2200).index                                          
data.loc[index.values,'Dis']=np.round(np.random.uniform(0.1,0.4,2200),decimals=2)      
index=data.loc[(data.Season==2017) & (data.Dis==0.01)].sample(4000).index
data.loc[index.values,'Dis']=np.round(np.random.uniform(0.1,0.4,4000),decimals=2)

In [7]:
data.loc[data.Season==2017,'Orientation']=np.mod(89.5+data.loc[data.Season==2017,'Orientation'],360)

In [8]:
df_rush=data.loc[data.Rusher]

### Relative distance and Speed to Rusher

In [9]:
%%time
#https://www.kaggle.com/jccampos/nfl-2020-winner-solution-the-zoo
# Getting Speed component along vertical and Horizontal Axis
data['S_x'] = data['S']*data['Dir_std'].apply(math.cos)
data['S_y'] = data['S']*data['Dir_std'].apply(math.sin)

#Momentum=mass*velocity along vertical and Horizontal Axis
data['M_x']=data.PlayerWeight*data['S_x']
data['M_y']=data.PlayerWeight*data['S_y']


rush=data[data.Rusher]
rush.set_index('PlayId',inplace=True,drop=True)
#Converting to Dictionary the values of Rusher
mapp=rush[['X_std','Y_std','S_x','S_y','M_x','M_y']].to_dict(orient='index')

#Creating columns which contain the rusher values to all PlayIds
rush_x=data['PlayId'].apply(lambda x:mapp[x]['X_std'])
rush_y=data['PlayId'].apply(lambda y:mapp[y]['Y_std'])
rush_Sx=data['PlayId'].apply(lambda x:mapp[x]['S_x'])
rush_Sy=data['PlayId'].apply(lambda y:mapp[y]['S_y'])
rush_Mx=data['PlayId'].apply(lambda x:mapp[x]['M_x'])
rush_My=data['PlayId'].apply(lambda y:mapp[y]['M_y'])



Wall time: 1.58 s


In [10]:
%%time
#Euclidean Distance between Rusher and other players
data['gap']=((rush_x-data['X_std'])**2+(rush_y-data['Y_std'])**2)**0.5
data['inverse_gap']=data['gap'].apply(lambda x: 1/x if x!=0 else -1)
#Relative Speeds between Rusher and other players
data['RelS_x']=rush_Sx-data['S_x']
data['RelS_y']=rush_Sy-data['S_y']
# Relative Momentum between Rusher and other players
data['RelM_x']=rush_Mx-data['M_x']
data['RelM_y']=rush_My-data['M_y']

Wall time: 275 ms


In [11]:
%%time

#Code to collapse all 22 players involved per PlayId into a single row arranged from highest to lowest.
playids=data.PlayId.unique()
A=data['gap'].values
B=data['RelS_x'].values
C=data['RelS_y'].values
D=data['inverse_gap'].values
E=data['RelM_x'].values
F=data['RelM_y'].values
n=len(playids)
val_dict={}
for i in tqdm(range(0,n)):
    val_dict[playids[i]]=sorted(A[i*22:i*22+22],reverse=True)+sorted(D[i*22:i*22+22])+\
                        sorted(B[i*22:i*22+22],reverse=True)+sorted(C[i*22:i*22+22],reverse=True)+\
                        sorted(E[i*22:i*22+22],reverse=True)+sorted(F[i*22:i*22+22],reverse=True)
val_cols=['P_{0}'.format(i+1) for i in range(0,22)]+['P_Invd_{0}'.format(i+1) for i in range(0,22)]+\
            ['Sx_{0}'.format(i+1) for i in range(0,22)]+['Sy_{0}'.format(i+1) for i in range(0,22)]+\
        ['Mx_{0}'.format(i+1) for i in range(0,22)]+['My_{0}'.format(i+1) for i in range(0,22)]

rel_data=pd.DataFrame(data=val_dict.values(),columns=val_cols,index=list(val_dict.keys()))
rel_data.head()

  0%|          | 0/31007 [00:00<?, ?it/s]

Wall time: 2.17 s


,P_1,P_2,P_3,P_4,P_5,P_6,P_7,P_8,P_9,P_10,P_11,P_12,P_13,P_14,P_15,P_16,P_17,P_18,P_19,P_20,P_21,P_22,P_Invd_1,P_Invd_2,P_Invd_3,P_Invd_4,P_Invd_5,P_Invd_6,P_Invd_7,P_Invd_8,P_Invd_9,P_Invd_10,P_Invd_11,P_Invd_12,P_Invd_13,P_Invd_14,P_Invd_15,P_Invd_16,P_Invd_17,P_Invd_18,P_Invd_19,P_Invd_20,P_Invd_21,P_Invd_22,Sx_1,Sx_2,Sx_3,Sx_4,Sx_5,Sx_6,Sx_7,Sx_8,Sx_9,Sx_10,Sx_11,Sx_12,Sx_13,Sx_14,Sx_15,Sx_16,Sx_17,Sx_18,Sx_19,Sx_20,Sx_21,Sx_22,Sy_1,Sy_2,Sy_3,Sy_4,Sy_5,Sy_6,Sy_7,Sy_8,Sy_9,Sy_10,Sy_11,Sy_12,Sy_13,Sy_14,Sy_15,Sy_16,Sy_17,Sy_18,Sy_19,Sy_20,Sy_21,Sy_22,Mx_1,Mx_2,Mx_3,Mx_4,Mx_5,Mx_6,Mx_7,Mx_8,Mx_9,Mx_10,Mx_11,Mx_12,Mx_13,Mx_14,Mx_15,Mx_16,Mx_17,Mx_18,Mx_19,Mx_20,Mx_21,Mx_22,My_1,My_2,My_3,My_4,My_5,My_6,My_7,My_8,My_9,My_10,My_11,My_12,My_13,My_14,My_15,My_16,My_17,My_18,My_19,My_20,My_21,My_22
20170907000118,22.415872,14.644511,13.296834,12.968593,12.041516,10.622476,9.902020,7.820038,7.500467,6.775161,6.480872,6.114785,5.448982,4.880256,4.789916,4.651075,4.593310,4.401931,4.364058,3.815337,1.449724,0.0,-1.0,0.044611,0.068285,0.075206,0.077109,0.083046,0.094140,0.100989,0.127877,0.133325,0.147598,0.154300,0.163538,0.183521,0.204907,0.208772,0.215004,0.217708,0.227173,0.229145,0.262100,0.689786,4.315577,4.094377,3.801623,3.713885,3.523219,3.392581,3.174778,2.870414,2.838041,2.635095,2.591315,2.453731,2.426079,2.306739,2.291043,2.069436,1.968706,1.662329,1.451470,1.232611,1.079220,0.000000,3.480931,2.977324,0.260698,0.196466,0.000000,-0.106316,-0.183183,-0.366237,-0.390181,-1.022786,-1.070726,-1.093659,-1.192375,-1.378258,-1.403284,-1.404518,-1.492353,-1.543467,-1.639462,-1.802612,-2.053748,-2.373170,422.739566,387.842045,353.722333,352.678873,327.902243,315.727782,290.141494,250.000780,249.616017,243.012877,231.268637,207.406314,199.178792,187.527310,187.206371,180.022042,152.984153,135.340706,120.584913,77.621167,46.477315,0.000000,334.956354,260.634352,54.178825,28.659156,27.812230,25.036121,23.628190,0.000000,-0.878790,-74.908264,-79.905003,-86.717851,-108.161121,-126.104842,-126.458872,-128.605904,-138.763738,-145.949821,-152.649005,-164.513588,-185.869813,-214.673542
20170907000139,23.025872,18.701749,18.102928,12.301711,11.383431,10.579986,10.224094,8.895802,8.583356,7.760058,6.658686,6.194837,5.821151,5.797525,5.322875,5.249019,4.287773,4.011646,3.787941,3.424617,0.792023,0.0,-1.0,0.043429,0.053471,0.055240,0.081290,0.087847,0.094518,0.097808,0.112413,0.116505,0.128865,0.150180,0.161425,0.171787,0.172487,0.187868,0.190512,0.233221,0.249274,0.263996,0.292003,1.262590,3.570393,3.219318,2.873532,2.778934,2.644771,2.641649,2.563632,2.555124,2.352256,1.861719,1.812925,1.326231,1.303935,1.108952,1.084985,0.623166,0.485740,0.369369,0.000000,-0.013551,-0.201967,-0.346848,5.653938,3.777791,3.538059,3.478843,3.401614,2.925481,2.831364,2.769456,2.707896,2.627165,2.423995,2.370496,1.911371,1.678202,1.408649,1.174073,0.970208,0.878559,0.260623,0.000000,-0.389860,-0.709247,327.267644,307.993152,282.326942,268.024843,251.171027,247.127501,245.807289,234.976622,220.548236,165.763632,155.193666,79.375083,68.835034,68.723401,59.524276,54.862064,47.256558,5.856340,0.000000,-20.338646,-38.179954,-135.729158,493.094309,345.033201,333.698684,319.356746,315.737377,279.922509,269.553668,261.146254,257.999066,250.396736,247.129498,232.284153,170.868664,139.793150,107.973426,95.778235,56.603746,39.512955,0.000000,-8.330682,-67.204392,-150.497135
20170907000189,20.726285,15.189486,14.379447,13.606149,11.708224,11.035959,10.485633,8.406343,8.387163,8.175292,7.560952,6.690030,6.203781,6.074084,5.697587,5.361576,4.982550,4.266017,4.221670,3.329625,1.646390,0.0,-1.0,0.048248,0.065835,0.069544,0.073496,0.085410,0.090613,0.095369,0.118958,0.119230,0.122320,0.132258,0.149476,0.161192,0.164634,0.175513,0.186512,0.200700,0.234411,0.236873,0.300334,0.607390,6.694662,4.793424,4.631415,4.202806,4.155699,4.062099,3.578924,3.504707,3.398847,3.183177,3.075629,2.635299,2.598447,2.458167,2.207957,2.090634,1.847263,1.737367,1.173281,0.794107,0.169864,0.000000,0.000

In [12]:
rel_data.index.rename('PlayId',inplace=True)

In [13]:
rel_data

,P_1,P_2,P_3,P_4,P_5,P_6,P_7,P_8,P_9,P_10,P_11,P_12,P_13,P_14,P_15,P_16,P_17,P_18,P_19,P_20,P_21,P_22,P_Invd_1,P_Invd_2,P_Invd_3,P_Invd_4,P_Invd_5,P_Invd_6,P_Invd_7,P_Invd_8,P_Invd_9,P_Invd_10,P_Invd_11,P_Invd_12,P_Invd_13,P_Invd_14,P_Invd_15,P_Invd_16,P_Invd_17,P_Invd_18,P_Invd_19,P_Invd_20,P_Invd_21,P_Invd_22,Sx_1,Sx_2,Sx_3,Sx_4,Sx_5,Sx_6,Sx_7,Sx_8,Sx_9,Sx_10,Sx_11,Sx_12,Sx_13,Sx_14,Sx_15,Sx_16,Sx_17,Sx_18,Sx_19,Sx_20,Sx_21,Sx_22,Sy_1,Sy_2,Sy_3,Sy_4,Sy_5,Sy_6,Sy_7,Sy_8,Sy_9,Sy_10,Sy_11,Sy_12,Sy_13,Sy_14,Sy_15,Sy_16,Sy_17,Sy_18,Sy_19,Sy_20,Sy_21,Sy_22,Mx_1,Mx_2,Mx_3,Mx_4,Mx_5,Mx_6,Mx_7,Mx_8,Mx_9,Mx_10,Mx_11,Mx_12,Mx_13,Mx_14,Mx_15,Mx_16,Mx_17,Mx_18,Mx_19,Mx_20,Mx_21,Mx_22,My_1,My_2,My_3,My_4,My_5,My_6,My_7,My_8,My_9,My_10,My_11,My_12,My_13,My_14,My_15,My_16,My_17,My_18,My_19,My_20,My_21,My_22
PlayId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20170907000118,22.415872,14.644511,13.296834,12.968593,12.041516,10.622476,9.902020,7.820038,7.500467,6.775161,6.480872,6.114785,5.448982,4.880256,4.789916,4.651075,4.593310,4.401931,4.364058,3.815337,1.449724,0.0,-1.0,0.044611,0.068285,0.075206,0.077109,0.083046,0.094140,0.100989,0.127877,0.133325,0.147598,0.154300,0.163538,0.183521,0.204907,0.208772,0.215004,0.217708,0.227173,0.229145,0.262100,0.689786,4.315577,4.094377,3.801623,3.713885,3.523219,3.392581,3.174778,2.870414,2.838041,2.635095,2.591315,2.453731,2.426079,2.306739,2.291043,2.069436,1.968706,1.662329,1.451470,1.232611,1.079220,0.000000,3.480931,2.977324,0.260698,0.196466,0.000000,-0.106316,-0.183183,-0.366237,-0.390181,-1.022786,-1.070726,-1.093659,-1.192375,-1.378258,-1.403284,-1.404518,-1.492353,-1.543467,-1.639462,-1.802612,-2.053748,-2.373170,422.739566,387.842045,353.722333,352.678873,327.902243,315.727782,290.141494,250.000780,249.616017,243.012877,231.268637,207.406314,199.178792,187.527310,187.206371,180.022042,152.984153,135.340706,120.584913,77.621167,46.477315,0.000000,334.956354,260.634352,54.178825,28.659156,27.812230,25.036121,23.628190,0.000000,-0.878790,-74.908264,-79.905003,-86.717851,-108.161121,-126.104842,-126.458872,-128.605904,-138.763738,-145.949821,-152.649005,-164.513588,-185.869813,-214.673542
20170907000139,23.025872,18.701749,18.102928,12.301711,11.383431,10.579986,10.224094,8.895802,8.583356,7.760058,6.658686,6.194837,5.821151,5.797525,5.322875,5.249019,4.287773,4.011646,3.787941,3.424617,0.792023,0.0,-1.0,0.043429,0.053471,0.055240,0.081290,0.087847,0.094518,0.097808,0.112413,0.116505,0.128865,0.150180,0.161425,0.171787,0.172487,0.187868,0.190512,0.233221,0.249274,0.263996,0.292003,1.262590,3.570393,3.219318,2.873532,2.778934,2.644771,2.641649,2.563632,2.555124,2.352256,1.861719,1.812925,1.326231,1.303935,1.108952,1.084985,0.623166,0.485740,0.369369,0.000000,-0.013551,-0.201967,-0.346848,5.653938,3.777791,3.538059,3.478843,3.401614,2.925481,2.831364,2.769456,2.707896,2.627165,2.423995,2.370496,1.911371,1.678202,1.408649,1.174073,0.970208,0.878559,0.260623,0.000000,-0.389860,-0.709247,327.267644,307.993152,282.326942,268.024843,251.171027,247.127501,245.807289,234.976622,220.548236,165.763632,155.193666,79.375083,68.835034,68.723401,59.524276,54.862064,47.256558,5.856340,0.000000,-20.338646,-38.179954,-135.729158,493.094309,345.033201,333.698684,319.356746,315.737377,279.922509,269.553668,261.146254,257.999066,250.396736,247.129498,232.284153,170.868664,139.793150,107.973426,95.778235,56.603746,39.512955,0.000000,-8.330682,-67.204392,-150.497135
20170907000189,20.726285,15.189486,14.379447,13.606149,11.708224,11.035959,10.485633,8.406343,8.387163,8.175292,7.560952,6.690030,6.203781,6.074084,5.697587,5.361576,4.982550,4.266017,4.221670,3.329625,1.646390,0.0,-1.0,0.048248,0.065835,0.069544,0.073496,0.085410,0.090613,0.095369,0.118958,0.119230,0.122320,0.132258,0.149476,0.161192,0.164634,0.175513,0.186512,0.200700,0.234411,0.236873,0.300334,0.607390,6.694662,4.793424,4.631415,4.202806,4.155699,4.062099,3.578924,3

## Combining Game state data to Player Data

In [14]:
final_df=pd.merge(df_rush,rel_data,on='PlayId')

In [15]:
df_rush.columns

Index(['GameId', 'PlayId', 'Team', 'S', 'A', 'Dis', 'Orientation', 'NflId',
       'DisplayName', 'JerseyNumber', 'Season', 'Quarter', 'GameClock',
       'PossessionTeam', 'Down', 'Distance', 'FieldPosition',
       'HomeScoreBeforePlay', 'VisitorScoreBeforePlay', 'NflIdRusher',
       'OffenseFormation', 'OffensePersonnel', 'DefendersInTheBox',
       'DefensePersonnel', 'PlayDirection', 'TimeHandoff', 'TimeSnap', 'Yards',
       'PlayerHeight', 'PlayerWeight', 'PlayerBirthDate', 'PlayerCollegeName',
       'Position', 'HomeTeamAbbr', 'VisitorTeamAbbr', 'Week', 'Stadium',
       'Location', 'StadiumType', 'Turf', 'GameWeather', 'Temperature',
       'Humidity', 'WindSpeed', 'WindDirection', 'Left', 'X_std', 'Y_std',
       'Rusher', 'Offense', 'YardLine_std', 'Dir_std'],
      dtype='object')

In [16]:
final_df.drop([ 'GameId','S','NflId','Team',
       'DisplayName', 'JerseyNumber', 'GameClock',
       'PossessionTeam', 'FieldPosition',
       'HomeScoreBeforePlay', 'VisitorScoreBeforePlay', 'NflIdRusher',
       'OffenseFormation', 'OffensePersonnel', 'DefendersInTheBox',
       'DefensePersonnel', 'PlayDirection', 'TimeHandoff', 'TimeSnap',
       'PlayerHeight', 'PlayerWeight', 'PlayerBirthDate', 'PlayerCollegeName',
       'Position', 'HomeTeamAbbr', 'VisitorTeamAbbr', 'Stadium',
       'Location', 'StadiumType', 'Turf','Left', 'X_std', 'Y_std',
       'Rusher', 'Offense', 'Dir_std','Season'],inplace=True,axis=1)


In [17]:
final_df.set_index('PlayId',inplace=True)

In [19]:
Y=np.array(final_df.Yards.values).reshape(-1,1)

In [20]:
def convert_cdf(yards):
    targets = yards
    test=np.zeros((yards.shape[0],199))
    for idx, t in tqdm(enumerate(list(yards))):
        test[idx][99 + t[0]] = 1
    return test
Y=convert_cdf(Y)


0it [00:00, ?it/s]

In [23]:
final_df.drop('Yards',axis=1,inplace=True)

In [20]:
del df_rush
del data
del rel_data

In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test=train_test_split(final_df,Y,test_size=0.35)

## Neural Network

In [22]:
import tensorflow as tf
import tensorflow.keras 
from tensorflow.keras.layers import Dense,Input,Dropout,BatchNormalization,Conv1D,AveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,LearningRateScheduler
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras import backend as K


class calls(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs):
        y_pred = self.model.predict(X_test)
        y_true = np.clip(np.cumsum(Y_test, axis=1), 0, 1)
        y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
        #y_pred=np.where(y_pred<0.1,0.0,1.0)

        val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * X_test.shape[0])
        print('val CRPS', val_s)
       
        
    
    
def crps_nn(y_true,y_pred):
    loss = K.mean(K.sum((K.cumsum(y_pred, axis = 1) - K.cumsum(y_true, axis=1))**2, axis=1))/199
    return loss
    
calls=calls()

def lrchange(epoch,lr):
    if (epoch+1)%3==0:
        return 0.9*lr
    else:
        return lr
    
lr=LearningRateScheduler(lrchange,verbose=1)
rl=ReduceLROnPlateau(monitor='val_crps_nn',min_delta=0.001,patience=3,verbose=1)
earlystop = EarlyStopping(monitor='val_crps_nn', min_delta=0.001, patience=10, verbose=1)

backs=[calls,lr,rl]
inputs=Input(shape=(X_train.shape[1],))
x=Dense(100,activation='relu')(inputs)
x=BatchNormalization()(x)
x=Dense(200,activation='relu')(x)
x=BatchNormalization()(x)
x=Dense(400,activation='relu')(x)
x=Dropout(0.5)(x)
out1=Dense(199,activation='softmax')(x)
model=Model(inputs,out1)
opt=Adam(0.005)
#loss=loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(%logoff)
model.compile(optimizer=opt,metrics=crps_nn,loss=crps_nn)
model.fit(x=X_train,y=Y_train,validation_data=(X_test,Y_test),batch_size=256,epochs=100,callbacks=backs)

Epoch 1/100

Epoch 00001: LearningRateScheduler reducing learning rate to 0.004999999888241291.
79/79 [==============================] - 2s 14ms/step - loss: 0.0170 - crps_nn: 0.0170 - val_loss: 0.0150 - val_crps_nn: 0.0150
val CRPS 0.0150208136794776
Epoch 2/100

Epoch 00002: LearningRateScheduler reducing learning rate to 0.004999999888241291.
79/79 [==============================] - 1s 11ms/step - loss: 0.0143 - crps_nn: 0.0143 - val_loss: 0.0143 - val_crps_nn: 0.0142
val CRPS 0.014255358338694667
Epoch 3/100

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0044999998994171625.
79/79 [==============================] - 1s 11ms/step - loss: 0.0140 - crps_nn: 0.0140 - val_loss: 0.0142 - val_crps_nn: 0.0142
val CRPS 0.014245895727868631
Epoch 4/100

Epoch 00004: LearningRateScheduler reducing learning rate to 0.0044999998062849045.
79/79 [==============================] - 1s 11ms/step - loss: 0.0139 - crps_nn: 0.0139 - val_loss: 0.0139 - val_crps_nn: 0.0139
val CRPS 0.013

In [24]:
model.save('nfl_pred.hdf5')

In [24]:
y_class=np.zeros((test.shape[0],1))
for idx,i in tqdm(enumerate(test)):
    y_class[idx]=int(np.where(i>0)[0][0])

X_train, X_test, Y_train, Y_test=train_test_split(final_df,y_class,test_size=0.25)

0it [00:00, ?it/s]

In [25]:
def crps_ml(y_true,y_pred):
    cr=((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * X_test.shape[0])
    return cr
def crps(model):
    y_true=Y_test
    y_pred=model.predict(X_test)
    y_pred=y_pred.reshape(-1,1)
    cr=((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * X_test.shape[0])
    return cr

## XGBoost

In [66]:
%%time
import warnings
warnings.filterwarnings("ignore")
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV


#xgb=XGBClassifier(tree_method='gpu_hist',)

parameters = {
'eta': [0.01, 0.1],
'n_estimators': [10,50,500],
'tree_method':['gpu_hist']   
}
#clf = GridSearchCV(xgb, parameters, cv=3, scoring='neg_root_mean_squared_error',return_train_score=True,verbose=3)
for i in parameters['eta']:
    for j in parameters['n_estimators']:
        xgb=XGBClassifier(eta=i,n_estimators=j,tree_method='gpu_hist',verbosity=1,eval_metric='mlogloss')
        xgb.fit(X_train,Y_train)
        print('params| eta:',i,'n_estimators',j,' validation CRPS',crps(xgb))



params| eta: 0.01 n_estimators 10  validation CRPS 0.22333157013135854
params| eta: 0.01 n_estimators 50  validation CRPS 0.22203769103515514
params| eta: 0.01 n_estimators 500  validation CRPS 0.22307810984748302
params| eta: 0.1 n_estimators 10  validation CRPS 0.22538583007918853
params| eta: 0.1 n_estimators 50  validation CRPS 0.22652996665473912
params| eta: 0.1 n_estimators 500  validation CRPS 0.22154503165984724
Wall time: 10min 58s


## Random Forest

In [70]:
%%time
from sklearn.ensemble import RandomForestClassifier

parameters = {'n_estimators': [10,50,100,500]}
#clf = GridSearchCV(xgb, parameters, cv=3, scoring='neg_root_mean_squared_error',return_train_score=True,verbose=3)
for i in parameters['n_estimators']:
    clf=RandomForestClassifier(n_estimators=i,n_jobs=-1)
    clf.fit(X_train,Y_train)
    print('n_estimators',i,' validation CRPS',crps(clf))


n_estimators 10  validation CRPS 0.25701456197395645
n_estimators 50  validation CRPS 0.234632916906514
n_estimators 100  validation CRPS 0.22837873578418408
n_estimators 500  validation CRPS 0.22512912861521228
Wall time: 1min 54s


## Decision Tree

In [31]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier


parameters = {'max_depth': [1,2,5,10,50]}
#clf = GridSearchCV(xgb, parameters, cv=3, scoring='neg_root_mean_squared_error',return_train_score=True,verbose=3)
for i in parameters['max_depth']:
    clf=DecisionTreeClassifier(max_depth=i)
    clf.fit(X_train,Y_train)
    print('max_depth',i,' validation CRPS',crps(clf))


max_depth 1  validation CRPS 0.23057949707256614
max_depth 2  validation CRPS 0.2148046735224108
max_depth 5  validation CRPS 0.22472527757466382
max_depth 10  validation CRPS 0.22460405743889728
max_depth 50  validation CRPS 0.3860342735348569


In [23]:
t=PrettyTable()
t.field_names=['Model','Best CRPS']
t.add_row(['Dense Neural Network',0.01356])
t.add_row(['XGBoost',0.22154])
t.add_row(['Random Forest',0.22512])
t.add_row(['Decision Tree',0.21480])
print(t)

+----------------------+-----------+
|        Model         | Best CRPS |
+----------------------+-----------+
| Dense Neural Network |  0.01356  |
|       XGBoost        |  0.22154  |
|    Random Forest     |  0.22512  |
|    Decision Tree     |   0.2148  |
+----------------------+-----------+
